In [5]:
import keras
import numpy as np
from keras.preprocessing import image

In [6]:
Tdata=image.load_img('../PIC/target.jpg')
Sdata=image.load_img('../PIC/style.png')
W,H=Tdata.size
print('target : '+str(W)+' - '+str(H))
W,H=Sdata.size
print('style : '+str(W)+' - '+str(H))

Targetdata=np.array(image.array_to_img(Tdata))
Targetdata=np.array([Targetdata])

Styledata=np.array(image.array_to_img(Sdata))
Styledata=np.array([Styledata])
print('target shape = '+str(Targetdata.shape))
print('style shape = '+str(Styledata.shape))

img_width,img_height=Tdata.size
print("image shape = "+str(Tdata.size))

target : 700 - 970
style : 550 - 765
target shape = (1, 970, 700, 3)
style shape = (1, 765, 550, 3)
image shape = (700, 970)


In [7]:
Tdata=image.load_img('../PIC/target.jpg')
img_width,img_height=Tdata.size
Sdata=image.load_img('../PIC/style.png',target_size=(img_height,img_width))
#타겟과 스타일의 shape을 맞춰야함. 섞은 이미지 텐서도 마찬가지임

#shape 확인
W,H=Tdata.size
print('target : '+str(W)+' - '+str(H))
W,H=Sdata.size
print('style : '+str(W)+' - '+str(H))

#이미지를 배열로 변환
Targetdata=np.array(image.array_to_img(Tdata))
Targetdata=np.array([Targetdata])

Styledata=np.array(image.array_to_img(Sdata))
Styledata=np.array([Styledata])

#shape = [사진 개수(?=1), H, W, RGB=3]
print('target shape = '+str(Targetdata.shape))
print('style shape = '+str(Styledata.shape))

target : 700 - 970
style : 700 - 970
target shape = (1, 970, 700, 3)
style shape = (1, 970, 700, 3)


In [8]:
#feature를 일정한 정도로 맞춰준다
Timg=np.array(Targetdata,dtype='float64')/10
Simg=np.array(Styledata,dtype='float64')/10
#턴세플로 레이어 생성
target = keras.backend.constant(Timg)
style = keras.backend.constant(Simg)

In [9]:
mix_img = keras.backend.placeholder((1, img_height, img_width, 3))

input_tensor=keras.backend.concatenate([target,style,mix_img], axis=0)

<h1>Definition of CNN Model</h1>
<h3>Input
<br>
<br>
Conv
<br>
Conv
<br>
MaxPooling
<br>
<br>
Conv
<br>
Conv
<br>
MaxPooling
<br>
<br>
Conv
<br>
Conv
<br>
MaxPooling
<br>
<br>
Conv
<br>
Conv
<br>
Conv
<br>
Conv
<br>
MaxPooling
<br>
<br>
Conv
<br>
Conv
<br>
Conv
<br>
Conv
<br>
MaxPooling
<br>
<br>
Dense
<br>
Dense
<br>
Dense

</h3>

In [12]:
#모델 로드하기. RESNET50 사용
#model = keras.applications.resnet50.ResNet50(input_tensor=input_tensor,weights='imagenet',include_top=False)
Input_layer_model = keras.layers.Input(shape=(970,700,3,))

x = keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu',padding='same',name='Conv01')(Input_layer_model)
x = keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu',padding='same',name='Conv02')(x)
x = keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)

x = keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu',padding='same',name='Conv11')(x)
x = keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu',padding='same',name='Conv12')(x)
x = keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)

x = keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu',padding='same',name='Conv21')(x)
x = keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu',padding='same',name='Conv22')(x)
x = keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)

x = keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same',name='Conv31')(x)
x = keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same',name='Conv32')(x)
x = keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same',name='Conv33')(x)
x = keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same',name='Conv34')(x)
x = keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)

x = keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same',name='Conv41')(x)
x = keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same',name='Conv42')(x)
x = keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same',name='Conv43')(x)
x = keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same',name='Conv44')(x)
x = keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)

x=keras.layers.Flatten()(x)
x = keras.layers.Dense(4096,activation='relu')(x)
x = keras.layers.Dense(4096,activation='relu')(x)

out_layer = keras.layers.Dense(1,activation='softmax')(x)

model_VGG=keras.models.Model(Input_layer_model,out_layer)

<h1>쓸모있는 함수들 정의. 주로 loss function.</h1>

In [22]:
model=model_VGG
import tensorflow as tf
#손실함수 정의
def content_loss(a,b):
    return keras.backend.sum(keras.backend.square(b-a))
#행렬 평면화 함수
def process_matrix(x):
    #(3)->(2,0,1)->(1,1,1)->(1)
    feature = keras.backend.batch_flatten(keras.backend.permute_dimensions(x,(2,0,1)))
    pro = keras.backend.dot(feature,keras.backend.transpose(feature))
def style_loss(og_style,gen_style):
    original=process_metrix(og_style)
    gen=process_metrix(gen_style)
    rgb=3
    size=img_height*img_width
    tmp=(4. * (rgb ** 2) * (size ** 2))
    return tf.sum(tf.square(original-gen)/tmp)
#모름.... 아마 함수 값을 더하는것 같은데 효율적으로 하려고 하이퍼 파라미터 조정을 하는듯.
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

<h2>어떤 레이어를 사용할건지 이름을 점함.</h2>
<h2>레이어와 레이어 이름을 해시맵에 넣고, 사용할 레이어 이름을 array로 정의한다.</h2>

In [26]:
#definition of layer name
resnet_layer=dict([(i.name,i.output) for i in model.layers])

#중요한 오류 발견. 레이어 이름은 실제 레이어에 있는 이름을 써야 함.
#따라서 Resnet50은 쓰기 어려움.

content_layer_name='Conv42'
style_layer_name=['Conv01',
                'Conv11',
                'Conv21',
                'Conv31',
                'Conv41']

In [25]:
# 웨이트 설정. 단지 최적화시킨 파라메터일뿐이다.
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = keras.backend.variable(0.)
layer_features = resnet_layer[content_layer_name]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features,combination_features)


In [129]:
model_VGG.

{'input_layers': [['input_19', 0, 0]],
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 970, 700, 3),
    'dtype': 'float32',
    'name': 'input_19',
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_19'},
  {'class_name': 'Conv2D',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'filters': 64,
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'distribution': 'uniform',
      'mode': 'fan_avg',
      'scale': 1.0,
      'seed': None}},
    'kernel_regularizer': None,
    'kernel_size': (3, 3),
    'name': 'Conv01',
    'padding': 'same',
    'strides': (1, 1),
    'trainable': True,
    'use_bias': True},
   'inbound_nodes': [[['input_19', 0, 0, {}]]],
   'name': 'Conv01'},